In [1]:
import os 
import sys
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC 
from sklearn.metrics import confusion_matrix, classification_report 

In [2]:
poses_sequence_path = '../../../Data/Fall/Dataset_CAUCAFall/Poses_sequences/'
list_of_files = os.listdir(poses_sequence_path)
list_of_files.sort()

In [3]:
df = pd.read_csv(poses_sequence_path + list_of_files[0], index_col=0)
df.head()

,X_nose,Y_nose,X_left_eye,Y_left_eye,X_right_eye,Y_right_eye,X_left_ear,Y_left_ear,X_right_ear,Y_right_ear,...,Y_right_hip,X_left_knee,Y_left_knee,X_right_knee,Y_right_knee,X_left_ankle,Y_left_ankle,X_right_ankle,Y_right_ankle,label
0,169.143264,185.373735,166.848344,180.783895,166.848344,178.488975,172.011914,176.767784,187.502626,160.129613,...,241.599281,255.202773,310.446889,272.414675,290.366337,274.135865,354.050373,285.610466,328.232521,Normal
1,179.250105,168.874884,178.000106,163.874887,178.625106,161.999889,184.875102,163.249888,205.500089,148.874897,...,252.624832,256.750057,309.499797,273.625047,292.624808,273.625047,352.624771,285.500039,329.499785,Normal
2,180.922243,168.069931,180.294801,162.422954,180.922243,161.168071,187.196661,162.422954,209.784566,147.364351,...,254.029460,257.470145,310.499224,273.783632,292.303411,273.783632,353.165268,285.705027,329.949920,Normal
3,180.909204,167.881737,180.269556,162.764552,181.548852,159.566312,187.305685,164.043849,209.053718,146.773352,...,254.234222,257.666969,310.523249,274.297818,293.252752,273.658170,352.740019,285.171834,329.712690,Normal
4,182.269733,165.962793,181.630085,160.845608,182.269733,157.647368,188.666214,162.124904,211.693543,145.494055,...,254.234222,257.108554,310.523249,273.739403,292.613104,273.739403,353.379667,285.253067,328.433394,Normal


## Modèle

In [4]:
import torch
import torch.nn as nn

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        out, _ = self.rnn(x, h0.detach())
        out = self.fc(out[:, -1, :])
        return out


## Test d'une passe forward

In [5]:
len(df)

126

In [6]:
def transform_labels(label):
    if label == 'Normal':
        return 0
    elif label == 'Fall':
        return 1
    else:
        return 2

In [7]:
# Prétraitement des données
sequence_length = 20
num_features = 34  # Si chaque membre a des positions X et Y

# Découper les séquences en fenêtres
sequences = [df.iloc[i:i+sequence_length, :-1].values for i in range(len(df) - sequence_length + 1)]
labels = [df['label'][i+sequence_length - 1] for i in range(len(df) - sequence_length + 1)]
labels = [transform_labels(label) for label in labels]
len(sequences), len(labels)

(107, 107)

In [8]:
# Normalisation
for i in range(len(sequences)):
    sequences[i] = (sequences[i] - sequences[i].mean()) / sequences[i].std()

# Conversion en tenseurs PyTorch
sequences = np.array(sequences)
sequences_tensor = torch.tensor(sequences, dtype=torch.float32)
sequences_tensor.shape

torch.Size([107, 20, 34])

In [9]:
classifier = RNNClassifier(input_size=num_features, hidden_size=32, num_layers=2, num_classes=3)
classifier(sequences_tensor)

tensor([[ 0.1023,  0.2418,  0.0149],
        [ 0.1316,  0.2738,  0.0291],
        [ 0.1268,  0.2846,  0.0201],
        [ 0.1257,  0.3109, -0.0028],
        [ 0.1062,  0.3364, -0.0570],
        [ 0.1396,  0.2820,  0.0063],
        [ 0.0640,  0.2887, -0.0810],
        [ 0.1069,  0.3284, -0.0615],
        [ 0.1171,  0.2489, -0.0300],
        [ 0.1127,  0.2136, -0.0351],
        [ 0.1470,  0.2492, -0.0489],
        [ 0.1605,  0.2579, -0.0614],
        [ 0.1561,  0.2575, -0.0577],
        [ 0.1604,  0.2490, -0.0636],
        [ 0.1605,  0.2564, -0.0738],
        [ 0.1622,  0.2589, -0.0799],
        [ 0.1553,  0.2695, -0.0983],
        [ 0.1581,  0.2709, -0.0994],
        [ 0.1537,  0.2644, -0.0988],
        [ 0.1641,  0.2678, -0.1004],
        [ 0.1659,  0.2647, -0.0996],
        [ 0.1731,  0.2794, -0.1313],
        [ 0.1732,  0.2714, -0.1332],
        [ 0.1566,  0.2422, -0.1114],
        [ 0.1514,  0.2442, -0.1166],
        [ 0.1444,  0.2304, -0.1009],
        [ 0.1478,  0.2441, -0.1055],
 

In [10]:
classifier(sequences_tensor[0:40])

tensor([[ 0.1023,  0.2418,  0.0149],
        [ 0.1316,  0.2738,  0.0291],
        [ 0.1268,  0.2846,  0.0201],
        [ 0.1257,  0.3109, -0.0028],
        [ 0.1062,  0.3364, -0.0570],
        [ 0.1396,  0.2820,  0.0063],
        [ 0.0640,  0.2887, -0.0810],
        [ 0.1069,  0.3284, -0.0615],
        [ 0.1171,  0.2489, -0.0300],
        [ 0.1127,  0.2136, -0.0351],
        [ 0.1470,  0.2492, -0.0489],
        [ 0.1605,  0.2579, -0.0614],
        [ 0.1561,  0.2575, -0.0577],
        [ 0.1604,  0.2490, -0.0636],
        [ 0.1605,  0.2564, -0.0738],
        [ 0.1622,  0.2589, -0.0799],
        [ 0.1553,  0.2695, -0.0983],
        [ 0.1581,  0.2709, -0.0994],
        [ 0.1537,  0.2644, -0.0988],
        [ 0.1641,  0.2678, -0.1004],
        [ 0.1659,  0.2647, -0.0996],
        [ 0.1731,  0.2794, -0.1313],
        [ 0.1732,  0.2714, -0.1332],
        [ 0.1566,  0.2422, -0.1114],
        [ 0.1514,  0.2442, -0.1166],
        [ 0.1444,  0.2304, -0.1009],
        [ 0.1478,  0.2441, -0.1055],
 

## Preprocessing

In [11]:
def transform_labels(label):
    if label == 'Normal':
        return 0
    elif label == 'Fall':
        return 1
    else:
        return 2

In [12]:
sequence_length = 20
num_features = 34

sequences = []
labels = []

for file in list_of_files:
    print("Processing file: ", file, end="\r")
    df = pd.read_csv(poses_sequence_path + file, index_col=0)
    for i in range(len(df) - sequence_length + 1):
        seq = df.iloc[i:i+sequence_length, :-1].values
        seq = (seq - seq.mean()) / seq.std()
        sequences.append(seq)
        labels.append(transform_labels(df['label'][i+sequence_length - 1]))

## Entraînement

In [13]:
import torch.nn.functional as F

sequences_tensor = torch.tensor(sequences, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

/tmp/ipykernel_61276/3821212035.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  sequences_tensor = torch.tensor(sequences, dtype=torch.float32)


In [14]:
sequences_tensor.shape

torch.Size([18101, 20, 34])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(sequences_tensor, labels_tensor, test_size=0.2, random_state=42)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
mini_batch_size = 16
n_epochs = 100

classifier = RNNClassifier(input_size=num_features, hidden_size=32, num_layers=2, num_classes=3)

for epoch in range(n_epochs):
    permutation = torch.randperm(X_train.size()[0])
    for i in range(0, X_train.size()[0], mini_batch_size):
        indices = permutation[i:i+mini_batch_size]
        mini_batch_x, mini_batch_y = X_train[indices], y_train[indices]
        optimizer.zero_grad()
        outputs = classifier(mini_batch_x)
        loss = criterion(outputs, mini_batch_y)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Minibatch : ', i, ', loss: ', loss.item())
    print('Epoch: ', epoch, ', loss: ', loss.item())

Epoch:  0 , loss:  1.0155349969863892
Epoch:  1 , loss:  nan
Epoch:  2 , loss:  0.9589605331420898
Epoch:  3 , loss:  1.0564699172973633
Epoch:  4 , loss:  1.011460781097412
Epoch:  5 , loss:  0.9891057014465332
Epoch:  6 , loss:  1.0225684642791748
Epoch:  7 , loss:  1.0270195007324219
Epoch:  8 , loss:  0.9875584840774536
Epoch:  9 , loss:  0.9833074808120728
Epoch:  10 , loss:  1.0450332164764404
Epoch:  11 , loss:  0.9929062724113464
Epoch:  12 , loss:  nan
Epoch:  13 , loss:  1.0593966245651245
Epoch:  14 , loss:  1.006899356842041
Epoch:  15 , loss:  1.0235196352005005
Epoch:  16 , loss:  1.032806634902954
Epoch:  17 , loss:  1.0033601522445679
Epoch:  18 , loss:  0.9984191656112671
Epoch:  19 , loss:  1.014947772026062
Epoch:  20 , loss:  1.0577119588851929
Epoch:  21 , loss:  0.9445362091064453
Epoch:  22 , loss:  1.0429261922836304
Epoch:  23 , loss:  1.032084345817566
Epoch:  24 , loss:  1.052648901939392
Epoch:  25 , loss:  nan
Epoch:  26 , loss:  1.0323525667190552
Epoch:  

KeyboardInterrupt: 